# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 64
NUM_EPOCHS = 20

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [4]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self, input_num=28*28, hidden_num_1=500, hidden_num_2=400, hidden_num_3=200, output_num=10):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_num, hidden_num_1), nn.BatchNorm1d(hidden_num_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(hidden_num_1, hidden_num_2), nn.BatchNorm1d(hidden_num_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(hidden_num_2, output_num))

    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images = images.view(images.size(0), -1)
        images = torch.autograd.Variable(images)
        labels = torch.autograd.Variable(labels)
        
        out = model(images)
        loss = criterion(out, labels)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()             
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
model.eval()
train_loss = 0
train_acc = 0
for images, labels in tqdm(train_loader):
    images = images.view(images.size(0), -1)
    images = torch.autograd.Variable(images)
    labels = torch.autograd.Variable(labels)
    out = model(images)
    loss = criterion(out, labels)
    train_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    train_acc += num_correct.item()

train_loss=train_loss / (len(train_dataset))
train_accuracy=train_acc / (len(train_dataset))

eval_loss = 0
eval_accurracy = 0
for images, labels in tqdm(test_loader):
    images = images.view(images.size(0), -1)
    images = torch.autograd.Variable(images)
    labels = torch.autograd.Variable(labels)  
    out = model(images)
    loss = criterion(out, labels)
    eval_loss += loss.data.item()*labels.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    eval_accurracy += num_correct.item()
    
test_loss=eval_loss / (len(test_dataset))
test_accuracy=eval_accurracy / (len(test_dataset))

print('Training accuracy: %0.2f%%' % (train_accuracy*100))
print('Testing accuracy: %0.2f%%' % (test_accuracy*100)) 

100%|██████████| 156/156 [00:02<00:00, 70.78it/s]

Training accuracy: 99.95%
Testing accuracy: 98.07%


#### Q5:
Please print the training and testing accuracy.